In [3]:
!pip install sklearn --upgrade
!pip install numpy --upgrade
!pip install pandas --upgrade


Requirement already up-to-date: sklearn in /opt/conda/envs/python2/lib/python2.7/site-packages
Requirement already up-to-date: scikit-learn in /opt/conda/envs/python2/lib/python2.7/site-packages (from sklearn)
Requirement already up-to-date: numpy>=1.8.2 in /opt/conda/envs/python2/lib/python2.7/site-packages (from scikit-learn->sklearn)
Requirement already up-to-date: scipy>=0.13.3 in /opt/conda/envs/python2/lib/python2.7/site-packages (from scikit-learn->sklearn)
You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: numpy in /opt/conda/envs/python2/lib/python2.7/site-packages
You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: pandas in /opt/conda/envs/python2/lib/python2.7/site-packages
Requirement already up-to-date: python-dateutil>=2.5.0

In [5]:
!pip install pip --upgrade

!conda install numpy 


Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 1.4MB 170kB/s ta 0:00:011
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | /

In [4]:
from pandas import DataFrame
import numpy as np
import pandas
from matplotlib import pyplot as plt

import string 
from scipy.stats import pearsonr
from scipy import stats
import statsmodels.formula.api as smf
import hdbscan
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', 20)

ImportError: Missing required dependencies ['numpy']

In [ ]:
# Read raw and bi-symmetric log transformed values
df_no_trans = pandas.read_csv("cell-ephyz-raw.csv")
df_all =      pandas.read_csv("cell-ephyz-transformed.csv")

#Set index to be the model ID
df_no_trans.set_index('Unnamed: 0', inplace=True)
df_all.set_index('Unnamed: 0', inplace=True)
df_no_trans.index.name = df_all.index.name = "Cell_ID"

#create default cluster path
df_all["ClusterPath"] = "/"
df_all["Cluster"] = -1
df_all["WasNoise"] = False

In [ ]:
prop_names = [
    'AP1Amplitude',
    'AP2Amplitude',
    'AP12AmplitudeDrop',
    'AP12AmplitudeChangePercent',
    'AP1SSAmplitudeChange',  
    
    'AP1WidthHalfHeight',
    'AP2WidthHalfHeight',
    'AP12HalfWidthChangePercent',
    
    'AP1WidthPeakToTrough',
    'AP2WidthPeakToTrough',
    
    'AP1RateOfChangePeakToTrough',
    'AP2RateOfChangePeakToTrough',    
    'AP12RateOfChangePeakToTroughPercentChange',
    
    'AP1AHPDepth',
    'AP2AHPDepth',
    'AP12AHPDepthPercentChange',
    
    'AP1DelayMean',
    'AP2DelayMean',
    
    'AP1DelaySD',
    'AP2DelaySD',
    
    'AP1DelayMeanStrongStim',
    'AP2DelayMeanStrongStim',
    
    'AP1DelaySDStrongStim',
    'AP2DelaySDStrongStim',
    
    'Burst1ISIMean',
    'Burst1ISIMeanStrongStim',
    
    'Burst1ISISD',
    'Burst1ISISDStrongStim',
    
    'InitialAccommodationMean',
    'SSAccommodationMean',
    'AccommodationRateToSS',
    'AccommodationAtSSMean',
    'AccommodationRateMeanAtSS',
    
    
    'ISIMedian',
    'ISICV',
    'ISIBurstMeanChange',
    
    'SpikeRateStrongStim',
    
    'InputResistance',
    
    'SteadyStateAPs',
    
    'FrequencyPassAbove',
    'FrequencyPassBelow',
    
    'RampFirstSpike',
]

In [ ]:
-


In [ ]:
PCA_and_Cluster(parent_path = "/", 
                hide_noise = True, 
                remove_noise = False, 
                k_means = False, 
                interactive=False,
                cluster_captions=['AS','npRB','RA','MS'],
                axis_captions=['PC1 ~Delay to 1st AP (StDevs)', 'PC2 ~2nd AP Amplitude Change (StDevs)', 'PC3 ~AP Width (StDevs)'])

In [ ]:
PCA_and_Cluster(parent_path = "/3/", 
                hide_noise = True, 
                remove_noise = True, 
                k_means = False, 
                interactive=False,
                cluster_captions=['awMS','rMS'],
                axis_captions=['PC1 ~Delay to 1st AP (StDevs)', 'PC2 ~Accommodation (StDevs)', 'PC3 ~AP Width (StDevs)'])

In [ ]:
%matplotlib notebook
%matplotlib notebook

In [ ]:
PCA_and_Cluster(parent_path = "/3/1/", 
                hide_noise = False, 
                remove_noise = True, 
                k_means = True, 
                interactive=False,
                cluster_captions=['aFS','dRS','naRS','RS','B','FS'],
                axis_captions=['PC1 ~Delay to 1st AP (StDevs)', 'PC2 ~Mean ISI (StDevs)', 'PC3 ~Accommodation (StDevs)'])

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib notebook
%matplotlib notebook

In [ ]:
# 3D plot of clusters in RAW feature space
source_df = df_all[df_all["ClusterPath"].str.startswith("/3/1/")]
#source_df = source_df[source_df["WasNoise"] == False]
clusters = source_df["Cluster"]

source_df = df_no_trans.ix[source_df.index]

cluster_captions=['aFS','dRS','naRS','RS','B','FS']
display_props = ["ISIMedian","AccommodationAtSSMean","AP1DelayMeanStrongStim"]
#display_props = ["AP1DelayMeanStrongStim","ISIMedian","AccommodationAtSSMean"]

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(
    source_df[display_props[0]],
    source_df[display_props[1]], 
    source_df[display_props[2]], 
    depthshade=False,
    marker='o', 
    c=clusters, 
    cmap='rainbow')

ax.set_xlabel("Median Interspike Interval (ms)")
ax.set_ylabel("Mean Accomodation at Steady State (%)")
ax.set_zlabel("Delay to 1st AP (ms)")

ax.set_xlim([0,250])
ax.set_ylim([-100,0])
ax.set_zlim([0,50])

labels = np.unique(clusters)

for i, l in enumerate(labels):
    X_label = source_df[clusters == l]
    center = [np.mean(X_label[prop]) for prop in display_props]
    ax.text(center[0],center[1],center[2],cluster_captions[i],size=20)

plt.tight_layout()

plt.show() 

In [ ]:
def plot_tetrahedron_projection(dim_x = 1, dim_y = 2, invert_x=False):
    %matplotlib inline

    ax_labels = ["Median Interspike Interval (ms)", 
                 "Mean Accomodation at Steady State (%)", 
                 "Delay to 1st AP (ms)"]

    fig = plt.figure()#figsize=(12, 10))
    ax = fig.add_subplot(111)

    ax.scatter(
        source_df[display_props[dim_x]],
        source_df[display_props[dim_y]], 
        marker='o', 
        c=clusters, 
        cmap='rainbow')

    ax.set_xlabel(ax_labels[dim_x])
    ax.set_ylabel(ax_labels[dim_y])

    labels = np.unique(clusters)

    for i, l in enumerate(labels):
        X_label = source_df[clusters == l]
        center = [np.mean(X_label[prop]) for prop in display_props]
        ax.text(center[dim_x],center[dim_y],cluster_captions[i],size=12,backgroundcolor="#2E917A",color="w",bbox={'linewidth':0,'alpha':0.75})
        

    plt.tight_layout()

    if invert_x:
        plt.gca().invert_xaxis()

    plt.show() 

In [ ]:
plot_tetrahedron_projection(0,1)
plot_tetrahedron_projection(0,2)
plot_tetrahedron_projection(1,2,True)

In [ ]:
#CLUSTER naRS RELATIONSHIP EXAMPLES
#display_props = ["ISIMedian","AccommodationAtSSMean","AP1DelayMeanStrongStim"]

min_x = source_df.ix[np.where(clusters == 1)]["AccommodationAtSSMean"].min()
print(source_df[source_df["AccommodationAtSSMean"] == min_x]["AP1DelayMeanStrongStim"])

max_x = source_df.ix[np.where(clusters == 1)]["AccommodationAtSSMean"].max()
print(source_df[source_df["AccommodationAtSSMean"] == max_x]["AP1DelayMeanStrongStim"])

In [ ]:
#CLUSTER dRS RELATIONSHIP EXAMPLES
#display_props = ["ISIMedian","AccommodationAtSSMean","AP1DelayMeanStrongStim"]

min_x = source_df.ix[np.where(clusters == 2)]["ISIMedian"].min()
print(source_df[source_df["ISIMedian"] == min_x]["AP1DelayMeanStrongStim"])

max_x = source_df.ix[np.where(clusters == 2)]["ISIMedian"].max()
print(source_df[source_df["ISIMedian"] == max_x]["AP1DelayMeanStrongStim"])

In [ ]:
from manager import ModelManager
from tables import Cells, Model_Waveforms, Models

import manager
reload(manager)
mgr = ModelManager()
mgr.server.connect()

In [ ]:
# Update database with cluster assignments
cell_records = list(Cells\
    .select(Cells,Model_Waveforms.Spikes,Models.Name)\
    .join(Model_Waveforms, on=(Cells.Model_ID == Model_Waveforms.Model_id))\
    .join(Models, on=(Cells.Model_ID == Models.Model_ID))\
    .where((Model_Waveforms.Protocol == "STEADY_STATE") & (Model_Waveforms.Variable_Name == "Voltage"))\
    .order_by(Cells.Model_ID)
)
   
for cell_id in df_all.index:
    cell = next(cell for cell in cell_records if cell.Model_ID == cell_id)
    cell.Ephyz_Cluster_ID = df_all.loc[cell_id]["ClusterPath"]
    
with mgr.server.db.atomic():
    for cell_id in df_all.index:
        cell = next(cell for cell in cell_records if cell.Model_ID == cell_id)
        cell.save()
        print(cell_id)
    
    print('saving...')